In [1]:
pip install -r ./requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.0/211.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.7 M

# Custom ChatGTT App with LangChain

In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break

    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)

Your prompt: What do you know about Lagos, Nigeria? 


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: What do you know about Lagos, Nigeria? 

> Finished chain.
{'content': 'What do you know about Lagos, Nigeria? ', 'text': 'Lagos is the largest city in Nigeria and one of the fastest-growing cities in the world. It is known for its vibrant culture, bustling markets, and lively music scene. Lagos is also a major financial center in Africa and is home to many multinational corporations. The city has a mix of modern skyscrapers and historic buildings, as well as beautiful beaches along the Atlantic Ocean. However, like many big cities, Lagos also faces challenges such as traffic congestion and inadequate infrastructure.'}
--------------------------------------------------
Your prompt: quit
Goodbye!


# Adding Chat Memory Using ConversationBufferMemory

In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False)

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)


memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)


prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break

    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)


Your prompt: Lagos is ....
{'content': 'Lagos is ....', 'chat_history': [HumanMessage(content='Lagos is ....'), AIMessage(content='Lagos is a vibrant and bustling city in Nigeria known for its vibrant culture, lively nightlife, and diverse population. It is also a major economic hub in Africa.')], 'text': 'Lagos is a vibrant and bustling city in Nigeria known for its vibrant culture, lively nightlife, and diverse population. It is also a major economic hub in Africa.'}
--------------------------------------------------
Your prompt: and its population is ...
{'content': 'and its population is ...', 'chat_history': [HumanMessage(content='Lagos is ....'), AIMessage(content='Lagos is a vibrant and bustling city in Nigeria known for its vibrant culture, lively nightlife, and diverse population. It is also a major economic hub in Africa.'), HumanMessage(content='and its population is ...'), AIMessage(content='Lagos has a population of over 14 million people, making it one of the largest citi

## Saving Chat Sessions

In [9]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False)

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break

    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)


Your prompt: how long does it take to travel from the sun to earth 
{'content': 'how long does it take to travel from the sun to earth ', 'chat_history': [HumanMessage(content='What is lightspeed in a vacuum?'), AIMessage(content='The speed of light in a vacuum is approximately 299,792 kilometers per second or about 186,282 miles per second. This is the fastest speed at which energy or information can travel in the universe.'), HumanMessage(content='what about the light speed in water?'), AIMessage(content='The speed of light in a medium like water is slower than in a vacuum. In water, the speed of light is about 225,000 kilometers per second or roughly 140,000 miles per second. This difference in speed is due to the interactions of light with the atoms and molecules in the medium, which can slow down the propagation of light.')], 'text': 'The average distance from the Sun to Earth is about 93 million miles (150 million kilometers). Since light travels at about 186,282 miles per second